In [ ]:
pwd

In [ ]:
!pip install keras==2.2.4
!pip install tensorflow==2.5.0

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen =ImageDataGenerator(rescale=1/255,zoom_range=0.2,horizontal_flip=True,vertical_flip=False)
test_datagen = ImageDataGenerator(rescale=1/255)

In [10]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='PVPkWFKW6iEOXHztR5TAl7h-ZiEWrzhAAJhndcHu_cq4',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'realtimecommunicationsystempowere-donotdelete-pr-hqyvwbjkomyjvl'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
if not hasattr(streaming_body_1,"__iter__"): streaming_body_1.__iter__= types.MethodType(__iter__,streaming_body_1)

In [11]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [12]:
x_train = train_datagen.flow_from_directory('/home/wsuser/work/Dataset/training_set',target_size=(64,64),batch_size=300,class_mode='categorical',color_mode='grayscale')

x_test = test_datagen.flow_from_directory('/home/wsuser/work/Dataset/test_set',target_size=(64,64),batch_size=300,class_mode='categorical',color_mode='grayscale')

Found 15750 images belonging to 9 classes.
Found 2250 images belonging to 9 classes.


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten

In [ ]:
model = Sequential()

In [16]:
model.add(Convolution2D(32,(3,3),input_shape=(64,64,1),activation='relu'))

In [17]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [18]:
model.add(Flatten())

In [19]:
model.add(Dense(units=512,activation='relu'))
model.add(Dense(units=9,activation='softmax'))

In [20]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit_generator(x_train,steps_per_epoch=24,epochs=10,validation_data=x_test,validation_steps=40)

model.save('aslpng1.h5')

In [ ]:
!tar -zcvf RealtimeModel.tgz signlanguage.h5

In [ ]:
!pip install watson-machine-learning-client --upgrade

In [23]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
 "url": "https://us-south.ml.cloud.ibm.com",
 "apikey": "GMWASXVsnkiFntXNKe_3ddzCruwAIJX7liRp6vx2Khc7"
}
client = APIClient(wml_credentials)

In [26]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return (next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [51]:
space_uid = guid_from_space_name(client, 'RealtimeModel')
print("Space UID : ", space_uid)

Space UID :  028cc4f6-7af4-4f1b-a0af-8ae4b7cbdfd9


In [52]:
client.set.default_space(space_uid)

'SUCCESS'

In [ ]:
client.software_specifications.list()

In [54]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")

In [55]:
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [58]:
model_details = client.repository.store_model(model='RealtimeModel.tgz',
meta_props={
 client.repository.ModelMetaNames.NAME: "CNN",
 client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
 client.repository.ModelMetaNames.TYPE: "tensorflow_2.7"})
model_id = client.repository.get_model_id(model_details)


In [ ]:
model_id

In [ ]:
client.repository.download(model_id,'aslpng1.tar.gz')

In [ ]:
ls